In [2]:
import jieba
import logging
import os
import sys
import multiprocessing

from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences

import warnings
import gensim

In [3]:
def seg_words():
    origin_file = "data/cleaned_tex.tex"  # 原始文件
    stop_words_file = "data/hit_stopwords.txt"  # 停用词路径
    user_dict_file = "data/jieba_dict.txt"  # 用户自定义词典路径
    stop_words = list()
    # 加载停用词
    with open(stop_words_file, 'r', encoding="utf8") as stop_words_file_object:
        contents = stop_words_file_object.readlines()
        for line in contents:
            line = line.strip()
            stop_words.append(line)
    # 加载用户字典(停用)
    jieba.load_userdict(user_dict_file)
    target_file = open("data/cut_word.txt", 'w', encoding="utf-8")
    with open(origin_file, 'r', encoding="utf-8") as origin_file_object:
        contents = origin_file_object.readlines()
        for line in contents:
            line = line.strip()
            out_str = ''
            word_list = jieba.cut(line, cut_all=False)
            for word in word_list:
                if word not in stop_words:
                    if word != "\t":
                        out_str += word
                        out_str += ' '
            target_file.write(out_str.rstrip() + "\n")
    target_file.close()

    print("end")
seg_words()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\12929\AppData\Local\Temp\jieba.cache
Loading model cost 0.721 seconds.
Prefix dict has been built successfully.


end


In [4]:
def train_model():
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(sys.argv))

    input_dir = 'data/cut_word.txt'
    outp1 = 'model/vector.model'
    outp2 = 'model/vector.vector'

    model = Word2Vec(PathLineSentences(input_dir),
                     size=100, min_count=2,
                     #window= 10,
                     workers=multiprocessing.cpu_count(), iter=10,sg=1)
    model.save(outp1)	# 存储二进制模型文件
    model.wv.save_word2vec_format(outp2, binary=False)	# 存储类似于数组的模型文件
train_model()

2023-06-09 19:26:50,885: INFO: running C:\Anaconda\envs\torch\lib\site-packages\ipykernel_launcher.py -f C:\Users\12929\AppData\Roaming\jupyter\runtime\kernel-911afc3c-7951-42cd-b5e5-62a7475759ea.json
2023-06-09 19:26:50,886: INFO: files read into PathLineSentences:data/cut_word.txt
C:\Anaconda\envs\torch\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(
2023-06-09 19:26:50,889: INFO: collecting all words and their counts
2023-06-09 19:26:50,890: INFO: reading file data/cut_word.txt
2023-06-09 19:26:50,896: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-09 19:26:50,901: INFO: collected 1512 word types from a corpus of 9111 raw words and 598 sentences
2023-06-09 19:26:50,902: INFO: Loading a fresh vocabulary
2023-06-09 19:26:50,906: INFO: effective_min_count=2 retains 840 unique words (55% of original 1512, drops 67

In [5]:
warnings.filterwarnings(action='ignore', category=UserWarning,module='gensim')
model = gensim.models.Word2Vec.load("model/vector.model")

word = '集合'
result = model.wv.most_similar(word)

print("跟 "+word+" 最相近的词：")
for i in result:
    print(i)

2023-06-09 19:27:27,317: INFO: loading Word2Vec object from model/vector.model
2023-06-09 19:27:27,332: INFO: loading wv recursively from model/vector.model.wv.* with mmap=None
2023-06-09 19:27:27,333: INFO: setting ignored attribute vectors_norm to None
2023-06-09 19:27:27,334: INFO: loading vocabulary recursively from model/vector.model.vocabulary.* with mmap=None
2023-06-09 19:27:27,334: INFO: loading trainables recursively from model/vector.model.trainables.* with mmap=None
2023-06-09 19:27:27,335: INFO: setting ignored attribute cum_table to None
2023-06-09 19:27:27,335: INFO: loaded model/vector.model
2023-06-09 19:27:27,337: INFO: precomputing L2-norms of word weight vectors


跟 集合 最相近的词：
('给定', 0.9978617429733276)
('求', 0.9965724349021912)
('二元运算', 0.9961625337600708)
('上', 0.9961559772491455)
('关系', 0.9953829646110535)
('子集', 0.9952859282493591)
('代数系统', 0.994737446308136)
('独异点', 0.9943711161613464)
('构成', 0.9940404295921326)
('判定', 0.9938764572143555)


In [6]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_product = norm(vector1) * norm(vector2)
    similarity = dot_product / norm_product
    return similarity

vec1 = model['运算'] - model['二元运算']
vec2 = model['度数'] - model['出度']

cosine_similarity(vec1,vec2)

C:\Users\12929\AppData\Local\Temp\ipykernel_12324\889666357.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec1 = model['运算'] - model['二元运算']
C:\Users\12929\AppData\Local\Temp\ipykernel_12324\889666357.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec2 = model['度数'] - model['出度']


0.26331678

In [7]:
result = model.wv.similar_by_vector(model['出度'] -  model['度数'] + model['无向图'],topn=5)
result

C:\Users\12929\AppData\Local\Temp\ipykernel_12324\1940268664.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  result = model.wv.similar_by_vector(model['出度'] -  model['度数'] + model['无向图'],topn=5)


[('无向图', 0.9965769052505493),
 ('人度', 0.9965161085128784),
 ('出度', 0.996300458908081),
 ('连通', 0.9958333969116211),
 ('有向图', 0.9952600598335266)]

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)
plt.show()


In [ ]:
from sklearn.decomposition import PCA

vectors = model.wv.vectors

# 计算任意两个词向量的差异
vector_differences = []
words = list(model.wv.vocab.keys())
for i in range(len(words)):
    for j in range(i + 1, len(words)):
        word1 = words[i]
        word2 = words[j]
        vector1 = vectors[i]
        vector2 = vectors[j]
        difference = vector1 - vector2
        vector_differences.append((word1+'-'+word2, difference))

#print(vector_differences[:10])

def reduce_dimensions_cha(vector_differences):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for item in vector_differences:
        vectors.append(item[1])
        labels.append(item[0])

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions_cha(vector_differences)

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)
plt.show()